Utilizando o Python para criar um script de carregamento para a "Plantas"

In [1]:
import pandas as pd

Mostrando o conteudo do CSV (Apenas os dois primeiros)

In [2]:
df = pd.read_csv('dim_plantas.csv')
df.head(2)

,id_planta,nome,pais,estado,cidade
0,1,NSK Brasil Ltda,Brazil,São Paulo,Suzano
1,2,NSK Argentina SRL,Argentina,Buenos Aires,Buenos Aires


Visualizando os metadados do nosso arquivo, para checagem de tipo de dados etc

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id_planta  10 non-null     int64 
 1   nome       10 non-null     object
 2   pais       10 non-null     object
 3   estado     10 non-null     object
 4   cidade     10 non-null     object
dtypes: int64(1), object(4)
memory usage: 532.0+ bytes


Primeiro modo, inserção manual, apenas cria um script de inserção.

In [5]:
with open('inserts_plantas.sql', 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        insert_sql = (
            "INSERT INTO plantas (id_planta, nome, pais, estado, cidade) "
            "VALUES ({}, '{}', '{}', '{}', '{}');\n".format(row['id_planta'], row['nome'], row['pais'], row['estado'], row['cidade'])
        )
        f.write(insert_sql)

print("Arquivo 'insert_plantas.sql' gerado com sucesso!")


Arquivo 'insert_plantas.sql' gerado com sucesso!


Segundo modo, inserção direta no banco de dados, executa os comandos de insert diretamente.

In [ ]:
# Primeiramente, vou utilizar uma biblioteca para conectar ao MySQL e inserir os dados do DataFrame.
import mysql.connector

# Função para inserir dados no MySQL, definindo o nome da tabela como 'linha_producao' e a conexão com o banco de dados.
def insert_data_to_mysql(df, table_name='plantas'):
    connection_params = {
        'host': 'localhost',
        'user': 'root',
        'password': 'pw4dummy',
        'database': 'prod_nsk'
    }
    data_to_insert = []  # declarar fora do try para retornar depois

    try:
        # Conectar ao banco de dados MySQL
        conn = mysql.connector.connect(**connection_params)
        # criar um cursor para executar comandos SQL
        cursor = conn.cursor()

        # SQL para inserir dados na tabela
        # Usando f-string para formatar a consulta SQL com o nome da tabela
        sql = f"INSERT INTO {table_name} (id_planta, nome, pais, estado, cidade) VALUES (%s, %s, %s, %s, %s)"

        # Iterar sobre o DataFrame e preparar os dados para inserção
        # Usando executemany para inserir múltiplas linhas de uma vez
        for _, row in df.iterrows():
            values = (
                int(row['id_planta']),
                row['nome'],
                row['pais'],
                row['estado'],
                row['cidade']
            )
            data_to_insert.append(values)

        cursor.executemany(sql, data_to_insert)
        conn.commit()

        #mensagem de sucesso
        print(f"{cursor.rowcount} registros inseridos com sucesso na tabela {table_name}.")
    
    except mysql.connector.Error as err:
        # mensagem de erro
        print(f"Erro MySQL: {err}")
    
    # fechar o cursor e a conexão
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
    
    return data_to_insert  # retornando aqui

# Executar e capturar os dados
data_to_insert = insert_data_to_mysql(df)


10 registros inseridos com sucesso na tabela plantas.
